# Developing New method for Using Kamodo with GEODYN

*This new method should run the orbits of GEODYN through Kamodo in a faster way that doesn't sacrifice accuracy of the interpolation in Kamodo and doesn't require a trimming of other Kamodo functionality to accomodate large overhead.*

1. Pre-run initialization of the orbit.
   - "initialize" a run of GEODYN with MSIS2 that gives us a "best guess" for the orbit of the satellite.
2. Construct box of estimated orbits with variation
   - Before running GEODYN with TIEGCM, we will take the "best guess" orbit and perturbate the input coordinates outward for each timestep that GEODYN asks for.  This will construct a box of best guesses around our satellite that can then be plugged into Kamodo.  
   - From here we just have a meshed index of possible density values from Kamodo for each timepoint and within some standard deviation of possible values along the orbit of the s/c.
3. Get density in GEODYN
   - now instead of calling the Kamodo model at each time step which has tons of overhead, we can call a relatively simple interpolation scheme in fortran at each timestep that will return the density value.  The orbit grid will be saved in a file that will be indexed by time according to what the MSIS model returned.
   - We will need to build in functionality such that if GEODYN requests a value outside of the box we requested, then it stops the program and restarts the process with a slightly larger grid around the orbit.
   
   


In [1]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import numpy as np

import pandas as pd
from tabulate import tabulate

## Function for trilinear interpolation

In [2]:

def interpolate_point_in_cube(lon, lat, alt, uncertain_grid, timestep, delta_deg, delta_m):
    '''
    This function performs a manual calculation for trilinear interoplation.
    Interpolates values on a cube to a desired coordinate.  
    Currently, it is set up for Lon, Lat, Alt.
    '''    
    x  = lon
    x0 = x - delta_deg
    x1 = x + delta_deg

    y  = lat
    y0 = y - delta_deg
    y1 = y + delta_deg

    z  = alt
    z0 = z - delta_m
    z1 = z + delta_m

    C000 = np.array(uncertain_grid[timestep]['values'][5])  # bottom, front, left
    C100 = np.array(uncertain_grid[timestep]['values'][7])  # bottom, front, right
    C001 = np.array(uncertain_grid[timestep]['values'][1])  # top,    front, left
    C101 = np.array(uncertain_grid[timestep]['values'][3])  # top,    front, right
    C010 = np.array(uncertain_grid[timestep]['values'][6])  # bottom, back,  left
    C110 = np.array(uncertain_grid[timestep]['values'][8])  # bottom, back,  right
    C011 = np.array(uncertain_grid[timestep]['values'][2])  # top,    back,  Left
    C111 = np.array(uncertain_grid[timestep]['values'][4])  # top,    back,  right


    ##===================================================================
    ##        Manually INTERPOLATE-- Trilinear Interpolation Calculation
    ##===================================================================

    ### On a periodic and cubic lattice, let xd, yd, zd be the differences 
    ### between each of x, y, z and the smaller coordinate related.
    #
    ###  x0 indicates the lattice point below x 
    ###  x1 indicates the lattice point above x
    xd =  (x-x0)/(x1-x0)
    yd =  (y-y0)/(y1-y0)
    zd =  (z-z0)/(z1-z0)
    # print(xd,yd,zd)

    ##### First we interpolate along x:   (push one of the X faces of cube towards the opposing face)
    ###   C000 uis the function value (x0, y0, z0)
    C00 = C000*(1-xd) + (C100*xd)
    C01 = C001*(1-xd) + (C101*xd)
    C10 = C010*(1-xd) + (C110*xd)
    C11 = C011*(1-xd) + (C111*xd)

    #### Next we interpolate along y:  (pushing the values that are now in middle of cube towards the center Y)
    C0 = C00*(1-yd) + C10*yd
    C1 = C01*(1-yd) + C11*yd

    #### Finally we interpolate along Z   (walk through the line that remains)
    C = C0*(1-zd) + C1*zd
    
    return(C)

## Do a Pre-run with MSIS2

In [3]:
# %load_ext autoreload
# %autoreload 2
# import copy
# import time
# import sys  


# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn


# ### Identify which arcs you want to run:
# arcs_files = [ 
#             '2018.336',  # 1
#              ]

# #------ A dictionary containing the run parameters ------  
# run_params = {} 
# run_params['arc']              =   arcs_files
# run_params['satellite']        =  'icesat2'  
# run_params['den_model']        =  'msis2'  
# run_params['SpecialRun_name']  =  'PCE_kamodo_interface'  
# run_params['verbose']          =  False


# %load_ext autoreload
# %autoreload 2

# import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn

# for imodel,val_model in enumerate( ['msis2']):
#     run_params1 = copy.deepcopy(run_params)
#     run_params1['den_model']  =  val_model  
#     run_params1['action']           =  'run'

#     ### Load the data into an object
#     Obj_Geodyn = Pygeodyn(run_params1)
#     Obj_Geodyn.RUN_GEODYN()
#     del Obj_Geodyn
    

## Read Density Values from Pre-Run

In [4]:


#### Need to bunzip the file we want to play with.

# !bunzip2 /data/data_geodyn/results/icesat2/msis2/msis2_acceloffPCE_kamodo_interface/DENSITY/icesat2_2018336_54hr.msis2.bz2

In [5]:
import sys

In [6]:

DEN_csv = pd.read_csv('/data/data_geodyn/results/icesat2/msis2/msis2_acceloffPCE_kamodo_interface/DENSITY/icesat2_2018336_54hr.msis2_msisin', 
#                     skiprows = 1, 
                    dtype=object,
                    names = ['YYMMDD',
                             'HHMMSS',
                             'Height_kilometers',
                             'Lat',
                             'Lon',
                             'STLOC',
                             'AVGFLX',
                             'FLUX',
                                 ],
                    sep = '\s+',
                    )


sat_time1 = list(DEN_csv['YYMMDD'])  #"031115" #  
sat_time2 = list(DEN_csv['HHMMSS'])  #"120000" #1068897600        
sattime   =    [x+y   for x,y   in zip(sat_time1, sat_time2)]

sattime   =    [datetime.strptime(x, '%y%m%d%H%M%S')   for x   in sattime ]
sattime   =    [datetime.timestamp(x)   for x   in sattime ]

DEN_csv['sattime_utctimestamp'] = sattime
# DEN_csv['Height_kilometers'] = DEN_csv['Height (meters)'].astype(float)*1e-3
DEN_csv['Lon'] = DEN_csv['Lon'].astype(float)
DEN_csv['Lat'] = DEN_csv['Lat'].astype(float)

timeHHMMSS = [] 
for i,val in enumerate(DEN_csv['HHMMSS'].values.astype(int)):
    # print(len(str(val)))
    if len(str(val)) == 1:
        timehhmmss_val = '00000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 2:
        timehhmmss_val = '0000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 3:
        timehhmmss_val = '000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 4:
        timehhmmss_val = '00'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 5:
        timehhmmss_val = '0'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    else:
        timeHHMMSS.append(str(val))
DEN_csv['timeHHMMSS'] = timeHHMMSS
YR = int(18)

YYMMDD_list = DEN_csv['YYMMDD'].astype(int).astype(str)
timeHHMMSS_list = DEN_csv['timeHHMMSS'].astype(str)

if YR < 10:
    year    = ['200' + x[:1]  for x in YYMMDD_list]
    month   = [        x[1:3] for x in YYMMDD_list]
    day     = [        x[3:]  for x in YYMMDD_list]
    hours   = [        x[:2]  for x in timeHHMMSS_list]
    minutes = [        x[2:4] for x in timeHHMMSS_list]
    secs    = [        x[4:]  for x in timeHHMMSS_list]
else:
    year    = ['20' + x[:2]  for x in YYMMDD_list]
    month   = [       x[2:4] for x in YYMMDD_list]
    day     = [       x[4:]  for x in YYMMDD_list]
    hours   = [       x[:2]  for x in timeHHMMSS_list]
    minutes = [       x[2:4] for x in timeHHMMSS_list]
    secs    = [       x[4:]  for x in timeHHMMSS_list]
#--------------------------------------------------------
DEN_csv['year']  = year
DEN_csv['month'] = month
DEN_csv['day']   = day
DEN_csv['hours']  = hours
DEN_csv['minutes'] = minutes
DEN_csv['secs']  = secs
#--------------------------------------------------------
year= list(map(int, DEN_csv['year'].values))
month= list(map(int, DEN_csv['month'].values))
day= list(map(int, DEN_csv['day'].values))
hour= list(map(int, DEN_csv['hours'].values))
minute = list(map(int, DEN_csv['minutes'].values))
second = list(map(int, DEN_csv['secs'].values))

DATE = list(map(datetime, year,month, day, hour,minute,second ))

#self.DEN_df['Date']  = DATE
DEN_csv.insert(0, 'Date', DATE)

del DEN_csv['timeHHMMSS']
del DEN_csv['year']
del DEN_csv['month']
del DEN_csv['day']
del DEN_csv['hours']
del DEN_csv['minutes']
del DEN_csv['secs']


DEN_csv

,Date,YYMMDD,HHMMSS,Height_kilometers,Lat,Lon,STLOC,AVGFLX,FLUX,sattime_utctimestamp
0,2018-12-01 21:01:09,181201,210109,486.242,41.976,65.686,1.55973,70.34,68.16,1.543698e+09
1,2018-12-01 21:02:09,181201,210209,485.539,38.151,65.208,1.54454,70.34,68.16,1.543698e+09
2,2018-12-01 21:03:09,181201,210309,484.882,34.323,64.753,1.53085,70.34,68.16,1.543698e+09
3,2018-12-01 21:04:09,181201,210409,484.290,30.492,64.316,1.51836,70.34,68.16,1.543698e+09
4,2018-12-01 21:05:09,181201,210509,483.779,26.659,63.892,1.50679,70.34,68.16,1.543698e+09
...,...,...,...,...,...,...,...,...,...,...
14353,2018-12-04 02:42:09,181204,024209,488.215,-16.097,-23.534,1.28048,70.27,68.67,1.543891e+09
14354,2018-12-04 02:43:09,181204,024309,489.508,-19.929,-23.932,1.27060,70.27,68.67,1.543891e+09
14355,2018-12-04 02:44:09,181204,024409,490.911,-23.759,-24.337,1.26023,70.27,68.67,1.543891e+09
14356,2018-12-04 02:45:09,181204,024509,492.405,-27.585,-24.752,1.24925,70.27,68.67,1.543892e+09


In [7]:
14357*9*4

516852

In [8]:
DEN_csv

,Date,YYMMDD,HHMMSS,Height_kilometers,Lat,Lon,STLOC,AVGFLX,FLUX,sattime_utctimestamp
0,2018-12-01 21:01:09,181201,210109,486.242,41.976,65.686,1.55973,70.34,68.16,1.543698e+09
1,2018-12-01 21:02:09,181201,210209,485.539,38.151,65.208,1.54454,70.34,68.16,1.543698e+09
2,2018-12-01 21:03:09,181201,210309,484.882,34.323,64.753,1.53085,70.34,68.16,1.543698e+09
3,2018-12-01 21:04:09,181201,210409,484.290,30.492,64.316,1.51836,70.34,68.16,1.543698e+09
4,2018-12-01 21:05:09,181201,210509,483.779,26.659,63.892,1.50679,70.34,68.16,1.543698e+09
...,...,...,...,...,...,...,...,...,...,...
14353,2018-12-04 02:42:09,181204,024209,488.215,-16.097,-23.534,1.28048,70.27,68.67,1.543891e+09
14354,2018-12-04 02:43:09,181204,024309,489.508,-19.929,-23.932,1.27060,70.27,68.67,1.543891e+09
14355,2018-12-04 02:44:09,181204,024409,490.911,-23.759,-24.337,1.26023,70.27,68.67,1.543891e+09
14356,2018-12-04 02:45:09,181204,024509,492.405,-27.585,-24.752,1.24925,70.27,68.67,1.543892e+09


## Make a cube of uncertainty around the init_orbit

**Considerations**
 - its possible that as long as our resolution is on par with MSIS we aren't introducing additional error. 
 - Gauss legendre sampling scheme
 - Perhaps can access geodyn's sampling method in the early iterations.
 - nearest neighbor interpolation within Fortran for optimized speed. Weighted average for distance between each point
 - how much memory to story the data-table in memory.
  - we will make MSISsubset  using msis


**Desired Features in the Data Structure**
 - Must be able to interpolate to a desired location
 - Should be able to expand the bounds of uncertainty at any given timestep along the orbit
    - prefereably without having to redo the already constructed portions
 - Each timestep should contain information on the `Lon`, `Lat`, `Alt` as well as the `RHO` and `DRHODZ` at each location
 - Include GEODYN run info (Satellite, Arc#, Arc Dates, density model used to initialize, etc.)


Parent: timestep  
    - each parent has square that contains uncertainty around the orbit 
    - lon lat, alt




In [9]:
# 20150*9

**START WITH A SIMPLE BOX**
Just draw a cube around the coordinate

## Next steps:

1. What data structure would work best for FORTRAN to read these tables at each timestep?
    - ~~linked list structue?~~
    - ~~a look up table?~~
    - ~~pass the dict straight to fortran?~~
    - NETCDF file
    - **CSV file lookup table**

2. Do a full run of the INIT_ORBIT through the Kamodo Cube of Uncertainty function
    - expand the deltas in each direction so that we always have our box surrounding the orbit.

3. Plug this data structure into fortran such that at any given timestep, requested by GEODYN the value can be found in the cube.

4. Add the interpolation function to the fortran code.

5. Do a full run of Kamodo-TIEGCM with the new interpolation method 
    - there is a chance this wont work and we may need to go ahead and implement the improved grid in the cube.
    
    
Additional Features and Queries that will need to be addressed:
  - increase the resolution of the grid within the cube
      - can maybe make it match MSISe2's grid resolution
      - if we have many cubes within the overall cube surrounding our INIT_ORBIT, we should write a simple selector code block that checks which sub-cube the coordinate is in, and only do the interpolation with that sub-cube.
  - determine by how much GEODYN perturbs the coordinates on the orbit in the early iterations
  
  - how can this all be written such that the file doesn't have to be re-written but is only added to.
      - can we construct the data-structures on a grid relative to earth such that additions can be made from separate runs?




### Write data as a simple CSV 

Just save each points time, value, coordinate, and corner on cube

In [10]:
TEST_NUMS = np.size(DEN_csv['Date'])
TEST_NUMS

14358

In [11]:
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough


times_list = []
lons_list = []
lats_list = []
alts_list = []
rhos_list = []
# drhodz_list = []
cube_corner = []

count=0
for it,val in enumerate(DEN_csv['Date'][:TEST_NUMS]):
    
    date_index = DEN_csv['YYMMDD'][it] + DEN_csv['HHMMSS'][it]
#     date_index = datetime.strptime(date_str, '%y%m%d%H%M%S')
    unix_time  = DEN_csv['sattime_utctimestamp'][it]
    print('****',date_index,'--',count, '****' )    
    count+=1

    ### Get the coordinates along the orbit:
    lon = float(DEN_csv['Lon'][it])
    lat = float(DEN_csv['Lat'][it])
    alt = float(DEN_csv['Height_kilometers'][it])
    center_coord = [lon, lat, alt]
    
    
    ### Find the coordinates of the cube surround the orbit point:
    delta_deg = 2    # degrees
    delta_m = 1000.*1e-3 # meters to kilometers
    A = [lon + delta_deg, lat+delta_deg, alt+delta_m]  # top,    front, left
    B = [lon + delta_deg, lat-delta_deg, alt+delta_m]  # top,    back,  Left
    C = [lon - delta_deg, lat+delta_deg, alt+delta_m]  # top,    front, right
    D = [lon - delta_deg, lat-delta_deg, alt+delta_m]  # top,    back,  right
    E = [lon + delta_deg, lat+delta_deg, alt-delta_m]  # bottom, front, left
    F = [lon + delta_deg, lat-delta_deg, alt-delta_m]  # bottom, back,  left
    G = [lon - delta_deg, lat+delta_deg, alt-delta_m]  # bottom, front, right
    H = [lon - delta_deg, lat-delta_deg, alt-delta_m]  # bottom, back,  right
    
    
    ### Store the cube's coordinates in the dictionary index
    cube_corners_and_center = []
    cube_corners_and_center.append(center_coord)
    cube_corners_and_center.append(A)
    cube_corners_and_center.append(B)
    cube_corners_and_center.append(C)
    cube_corners_and_center.append(D)
    cube_corners_and_center.append(E)
    cube_corners_and_center.append(F)
    cube_corners_and_center.append(G)
    cube_corners_and_center.append(H)
    
    #### Import Coordinates to Kamodo
    ##
    #### Kamodo static inputs:
    model          = 'TIEGCM'
    file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Lutz_Rastaetter_072319_IT_1/'
    variable_list  = ['rho','psi_O2', 'psi_O',  'psi_He', 'T_n']
    coord_type     = 'SPH'
    coord_grid     = 'sph'
    high_res       = 1.
    verbose        = False  
    csv_output     = '' 
    plot_output    = ''
    
    
    #### Extract the coordinates from each list to plug into Kamodo with vectorization
    lons_in = [item[0] for item in cube_corners_and_center]
    lats_in = [item[1] for item in cube_corners_and_center]
    alts_in = [item[2] for item in cube_corners_and_center]
       
    
    ## Gather inputs
    sat_time       = unix_time*np.ones(np.size(alts_in))
    c1             = lons_in
    c2             = lats_in
    c3             = alts_in
    
    ## Plug vectorized coordinates into Kamodo
    results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                        coord_type, coord_grid, high_res=20., verbose=False, 
                        csv_output='', plot_output='')

    
    
    corners = ['0','1','2','3','4','5','6','7','8']
    for ival, valrho in enumerate(results['rho']):
#         print(ival)
        times_list.append(date_index)
        lons_list.append(results['c1'][ival])
        lats_list.append(results['c2'][ival])
        alts_list.append(results['c3'][ival])
        rhos_list.append(valrho)
#         drhodz_list.append( )
        cube_corner.append(corners[ival])
    

filename ='/data/data_geodyn/atmos_models_data/tiegcm/2018/Lutz_Rastaetter_072319_IT_1/orbit_step60_w_UncCub_csv.csv'

# pd.options.display.float_format = '${:,.6f}'.format

df = pd.DataFrame(data={
                        'times':times_list  ,
                        'lon':lons_list  ,
                        'lat':lats_list  ,
                        'alt':alts_list  ,
                        'rho':rhos_list  ,
                        'corner':cube_corner  ,
                        })

file = open(filename,  'w')
for i,val in enumerate(df['times']):
    file.write(f"{df['times'][i]}   {df['lon'][i]:8.4f}   {df['lat'][i]:8.4f}   {df['alt'][i]:8.4f}   {df['rho'][i]:15.8e}   {df['corner'][i]} \n")


file.close()




import time: 1.3782889880239964
**** 181201210109 -- 0 ****
**** 181201210209 -- 1 ****
**** 181201210309 -- 2 ****
**** 181201210409 -- 3 ****
**** 181201210509 -- 4 ****
**** 181201210009 -- 5 ****
**** 181201205909 -- 6 ****
**** 181201205809 -- 7 ****
**** 181201205709 -- 8 ****
**** 181201205609 -- 9 ****
**** 181201210509 -- 10 ****
**** 181201210409 -- 11 ****
**** 181201210309 -- 12 ****
**** 181201210209 -- 13 ****
**** 181201210009 -- 14 ****
**** 181201205909 -- 15 ****
**** 181201205809 -- 16 ****
**** 181201205709 -- 17 ****
**** 181201205609 -- 18 ****
**** 181201210509 -- 19 ****
**** 181201210409 -- 20 ****
**** 181201210309 -- 21 ****
**** 181201210209 -- 22 ****
**** 181201210009 -- 23 ****
**** 181201205909 -- 24 ****
**** 181201205809 -- 25 ****
**** 181201205709 -- 26 ****
**** 181201205609 -- 27 ****
**** 181201210509 -- 28 ****
**** 181201210409 -- 29 ****
**** 181201210309 -- 30 ****
**** 181201210209 -- 31 ****
**** 181201210009 -- 32 ****
**** 181201205909 -- 

KeyboardInterrupt: 

In [ ]:
# results

## Compare indexed rho to pymsis


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from pymsis import msis
# import pandas as pd
# import sys
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SingleSatelliteFlythrough import SingleModelFlythrough
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SatelliteFlythrough import ModelFlythrough


# # import pandas as pd
# msisin_den_file = '/data/data_geodyn/results/st/msis2/msis2_acceloffSLR_kamodo_interface/DENSITY/' + "st031109_2wk.goco05s_msisin"
# # DEN_csv = pd.read_csv('/data/data_geodyn/results/icesat2/msis2/msis2_acceloffPCE_kamodo_interface/DENSITY/icesat2_2018336_54hr.msis2_msisin', 
# # #                     skiprows = 1, 
# #                     dtype=object,
# #                     names = ['YYMMDD',
# #                              'HHMMSS',
# #                              'Height_kilometers',
# #                              'Lat',
# #                              'Lon',
# #                              'STLOC',
# #                              'AVGFLX',
# #                              'FLUX',
# #                                  ],
# #                     sep = '\s+',
# #                     )

# DEN1_csv = pd.read_csv(msisin_den_file, 
#                     skiprows = 1, 
#                     names = ['IYYDDD',
#                              'IYR',
#                               'DAY',
#                              'UTSEC',
#                              'ALTKM',
#                              'GLAT',
#                              'GLON',
#                              'STLOC', 
#                              'AVGFLX',
#                              'FLUX',
#                              'AP1',
#                              'AP2',
#                              'AP3',
#                              'AP4',
#                              'AP5',
#                              'AP6',
#                              'AP7',
#                             ],
#                     sep = '\s+',
#                     )


# DEN1_csv['Date'] = (pd.to_datetime('0'+ ((DEN1_csv['IYR'].astype(int).astype(str))),  format='%y') 
#                     +  pd.to_timedelta(DEN1_csv['DAY'], unit='days'))



# SWI_option = [1.0]*25
# SWI_option[8] = -1.0




# msis2_df= pd.DataFrame(data={'Rho'   :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'N2'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'O2'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'O'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'He'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'H'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'Ar'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'N'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'AnomO' :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'NO'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
# #                               'Temp'  :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                             })


# for index, row in DEN1_csv[:500].iterrows():

#     lon = DEN_csv['Lon'][index] #row['GLON']
#     lat = DEN_csv['Lat'][index] #row['GLAT']
#     alts = DEN_csv['Height_kilometers'][index] #row['ALTKM']   
#     f107 = DEN_csv['FLUX'][index] # row['FLUX'] 
#     f107a = DEN_csv['AVGFLX'][index] # row['AVGFLX']

#     aps = [[row['AP1'],row['AP2'],row['AP3'],row['AP4'],row['AP5'],row['AP6'],row['AP7']]]
#     date = DEN_csv['Date'][index]  #row['Date']

#     msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option)
#     msis_data2  = np.squeeze(msis_data2)

#     msis2_df.loc[index, 'Rho'] = msis_data2[0]
#     msis2_df.loc[index, 'Date'] = row['Date']


    
    
#     #### Kamodo static inputs:
#     model          = 'TIEGCM'
#     file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Lutz_Rastaetter_072319_IT_1/'
#     variable_list  = ['rho','psi_O2', 'psi_O',  'psi_He', 'T_n']
#     coord_type     = 'SPH'
#     coord_grid     = 'sph'
#     high_res       = 1.
#     verbose        = False  
#     csv_output     = '' 
#     plot_output    = ''
    
           
#     unix_time  = DEN_csv['sattime_utctimestamp'][index]
# #                 DEN_csv['Lon'][index]
#     ## Gather inputs
#     sat_time       = unix_time*np.ones(np.size(lon))
#     c1             = [float(lon)]
#     c2             = [float(lat)]
#     c3             = [float(alts)]
    
#     ## Plug vectorized coordinates into Kamodo
#     results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
#                         coord_type, coord_grid, high_res=20., verbose=False, 
#                         csv_output='', plot_output='')


    
    
# #################### MAKE PLOT ####################

# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px


# config = dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })

# fig = make_subplots(rows=1, cols=1,
#                    )
# fig.add_trace(go.Scatter(x=msis2_df['Date'][:500],
#                          y=msis2_df['Rho'][:500],
#                          name= 'MSIS2 direct read',
#                          mode='markers',
#                          marker=dict(size=8)),
#                          )
# fig.add_trace(go.Scatter(y=DEN_csv['rho (kg/m**3)'][:500],
#                          x=DEN_csv['Date'][:500],
#                          name= 'Density File (GEODYN_msis2)',
#                          mode='markers',
#                         marker=dict(size=4)),
#                          )
                        
# # fig.add_trace(go.Scatter(y=results['max_height']*1e-3, x=results['rho'], mode='markers'))

# fig.update_yaxes(type="log", exponentformat= 'power', row=1, col=1, title ='Den')
# fig.update_xaxes(title ='Date')
# fig.update_layout(title="MSIS2 Direct Read, Comparison of Values")

# fig.show()